In [1]:
# %pip install pipenv
# %pip install scikit-learn
# %pip install seaborn
# %pip install matplotlib
# %pip install numpy
# %pip install tensorflow
# %pip install xgboost
# %pip install ipympl
%matplotlib widget


In [163]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, FunctionTransformer, PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


In [164]:
train_dataset = pd.read_csv('train.csv')

In [165]:
X_all = train_dataset.iloc[:,:-1]
Y_all = train_dataset.iloc[:,-1]
X_train, X_validate, Y_train, Y_validate = train_test_split(X_all, Y_all, test_size=0.2)
print(X_train.shape, Y_train.shape, X_validate.shape, Y_validate.shape)

(1168, 80) (1168,) (292, 80) (292,)


In [209]:
def check_model(X_train, Y_train, X_validate, Y_validate, model, preprocessor):
    print(X_train.shape, Y_train.shape)
    preprocessor.fit(X_train)
    X_train_scaled = preprocessor.transform(X_train)
    print('X_train_scaled shape', X_train_scaled.shape)
    model.fit(X_train_scaled, Y_train)
    Y_train_predict = model.predict(X_train_scaled)
    train_r2score = r2_score(Y_train, Y_train_predict)
    print(f'Train R2={train_r2score:.3f}')

    X_validate_scaled = preprocessor.transform(X_validate)
    Y_validate_predict = model.predict(X_validate_scaled)
    validate_r2score = r2_score(Y_validate, Y_validate_predict)
    print(f'Validate R2={validate_r2score:.3f}')

    return validate_r2score


In [198]:
def build_preprocessor():
    return make_pipeline(
        ColumnTransformer(transformers=[
           ('numeric1', StandardScaler(), ['LotArea', 'GrLivArea', 'OverallQual', 'YearBuilt'])
        ])
    )

def build_model():
    model = LinearRegression()
    return model

check_model(X_train, Y_train, X_validate, Y_validate, build_model(), build_preprocessor())

(1168, 80) (1168,)
X_train_scaled shape (1168, 4)
Train R2=0.747
Validate R2=0.751


In [199]:
def build_preprocessor_2():
    return make_pipeline(
        ColumnTransformer(transformers=[
           ('numeric1', StandardScaler(), 
            ['LotArea', 'GrLivArea', 'OverallQual', 'YearBuilt', 'GarageArea', 'Fireplaces']
            )
        ])
    )

check_model(X_train, Y_train, X_validate, Y_validate, build_model(), build_preprocessor_2())

(1168, 80) (1168,)
X_train_scaled shape (1168, 6)
Train R2=0.761
Validate R2=0.782


In [237]:
def build_preprocessor_3():
    return make_pipeline(
        ColumnTransformer(transformers=[
           ('numeric1', StandardScaler(), ['OverallQual', 'YearBuilt', 'Fireplaces', 'GarageArea']),
           ('numeric_log', FunctionTransformer(lambda x: np.log(x+0.001)), ['LotArea', 'GrLivArea'])
        ])
    )

check_model(X_train, Y_train, X_validate, Y_validate, build_model(), build_preprocessor_3())

(1168, 80) (1168,)
X_train_scaled shape (1168, 6)
Train R2=0.753
Validate R2=0.773


0.7725482285776857

In [238]:
selected_numerical_features = [
    'OverallQual', 'YearBuilt', 'Fireplaces', 'GarageArea',
    'LotArea', 'GrLivArea',
    'YearRemodAdd', 'TotRmsAbvGrd',
    'WoodDeckSF', 
]

def build_preprocessor_3a():
    return make_pipeline(
        ColumnTransformer(transformers=[
           ('numeric1', StandardScaler(), selected_numerical_features),
        ])
    )

check_model(X_train, Y_train, X_validate, Y_validate, build_model(), build_preprocessor_3a())

(1168, 80) (1168,)
X_train_scaled shape (1168, 9)
Train R2=0.766
Validate R2=0.798


0.7980387649173375

In [239]:
print('selected_numerical_features',selected_numerical_features)
print('one_hot_features', one_hot_features)
print('ordinal_features', ordinal_features)

selected_numerical_features ['OverallQual', 'YearBuilt', 'Fireplaces', 'GarageArea', 'LotArea', 'GrLivArea', 'YearRemodAdd', 'TotRmsAbvGrd', 'WoodDeckSF']
one_hot_features ['MSZoning', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'BldgType', 'HouseStyle', 'GarageFinish', 'SaleType']
ordinal_features ['ExterQual', 'ExterCond']


## Try with scaling output

In [ ]:
def build_preprocessor_5():
    return make_pipeline(
        ColumnTransformer(transformers=[
           ('numeric1', StandardScaler(), selected_numerical_features),
        ])
    )

check_model(X_train, np.log(Y_train), X_validate, np.log(Y_validate), build_model(), build_preprocessor_5())

(1168, 80) (1168,)
X_train_scaled shape (1168, 9)
Train R2=0.830
Validate R2=0.814


0.8138326612804762

## Enhance with one hot encoded features

In [ ]:
one_hot_features = [
'MSZoning',
'LotShape',
'LandContour',
'LotConfig',
'LandSlope',
'Neighborhood',
'BldgType',
'HouseStyle',
# 'RoofStyle',
# 'Exterior1st',
# 'Exterior2nd',
# 'Foundation',
# 'CentralAir',
# 'Electrical',
# 'GarageType',
'GarageFinish',
'SaleType',
# 'SaleCondition',
# 'RoofMatl' #Missing ['ClyTile', 'Metal'] in train data
# 'BldgType'
]

def build_preprocessor_with_onehot_1():
    return make_pipeline(
        ColumnTransformer(transformers=[
           ('numeric1', StandardScaler(), selected_numerical_features),
           ('one_hot', OneHotEncoder(), one_hot_features)
        ])
    )

check_model(X_train, Y_train, X_validate, Y_validate, build_model(), build_preprocessor_with_onehot_1())

(1168, 80) (1168,)
X_train_scaled shape (1168, 81)
Train R2=0.843
Validate R2=0.834


0.8341396345955728

## Enhance with ordinal features

In [242]:
ordinal_values_order = ['Po', 'Fa', 'TA', 'Gd', 'Ex']
ordinal_features = [
    'ExterQual',
    'ExterCond',
    # 'HeatingQC',
    # 'KitchenQual'
]
ordinal_value_per_feature = len(ordinal_features)*[ordinal_values_order]

def build_preprocessor_with_ordinal():
    numeric_transformer = make_pipeline(
        SimpleImputer(strategy='mean'),
        StandardScaler()
    )
    ordinal_encoder = make_pipeline(
        OrdinalEncoder(categories=ordinal_value_per_feature, 
                                      encoded_missing_value=ordinal_values_order.index('TA')),
        StandardScaler()
    )
    one_hot_encoded = make_pipeline(
        SimpleImputer(strategy='most_frequent'), OneHotEncoder()
    )
    return make_pipeline(
        ColumnTransformer(transformers=[
           ('numeric1', numeric_transformer, selected_numerical_features),
           ('one_hot', one_hot_encoded, one_hot_features),
           ('ordinal', ordinal_encoder, ordinal_features),
        ])
    )

check_model(X_train, Y_train, X_validate, Y_validate, build_model(), build_preprocessor_with_ordinal())

(1168, 80) (1168,)
X_train_scaled shape (1168, 82)
Train R2=0.844
Validate R2=0.842


0.8421180857827473

### Test with MasVnrArea

In [243]:
def build_preprocessor_with_masvnrarea():
    numeric_transformer = make_pipeline(
        SimpleImputer(strategy='mean'),
        StandardScaler()
    )
    numeric_transformer_2 = make_pipeline(
        SimpleImputer(strategy='constant', fill_value=0),
        StandardScaler() 
    )
    ordinal_encoder = make_pipeline(
        OrdinalEncoder(categories=ordinal_value_per_feature, 
                                      encoded_missing_value=ordinal_values_order.index('TA')),
        StandardScaler()
    )
    one_hot_encoded = make_pipeline(
        SimpleImputer(strategy='most_frequent'), OneHotEncoder()
    )
    return make_pipeline(
        ColumnTransformer(transformers=[
           ('numeric1', numeric_transformer, selected_numerical_features),
           ('numeric2', numeric_transformer_2, ['MasVnrArea']),
           ('one_hot', one_hot_encoded, one_hot_features),
           ('ordinal', ordinal_encoder, ordinal_features),
        ])
    )

check_model(X_train, Y_train, X_validate, Y_validate, build_model(), build_preprocessor_with_masvnrarea())

(1168, 80) (1168,)
X_train_scaled shape (1168, 83)
Train R2=0.845
Validate R2=0.847


0.8468366392200409

## Check model with polynomials

In [254]:
def build_preprocessor_4(degree):
    numeric_transformer = make_pipeline(
        SimpleImputer(strategy='mean'),
        StandardScaler(),
        PolynomialFeatures(degree=degree, include_bias=False),
        StandardScaler(with_mean=False)
    )
    numeric_transformer_2 = make_pipeline(
        SimpleImputer(strategy='constant', fill_value=0),
        StandardScaler() 
    )
    ordinal_encoder = make_pipeline(
        OrdinalEncoder(categories=ordinal_value_per_feature, 
                                      encoded_missing_value=ordinal_values_order.index('TA')),
        StandardScaler()
    )
    one_hot_encoded = make_pipeline(
        SimpleImputer(strategy='most_frequent'), OneHotEncoder()
    )
    return make_pipeline(
        ColumnTransformer(transformers=[
           ('numeric1', numeric_transformer, selected_numerical_features),
           ('numeric2', numeric_transformer_2, ['MasVnrArea']),
           ('one_hot', one_hot_encoded, one_hot_features),
           ('ordinal', ordinal_encoder, ordinal_features),
        ])
    )

def model_2(alpha):
    return Ridge(alpha=alpha)

best_params = (None, None)
best_result = None

for degree in range(2, 4):
    alpha = 3
    print(f'Degree={degree}, alpha={alpha}')
    preprocessor = build_preprocessor_4(degree)
    model = model_2(alpha)
    score = check_model(X_train, Y_train, X_validate, Y_validate, model, preprocessor)
    if not best_result or score > best_result:
        best_result = score
        best_params = (degree, alpha)
    print('---')

all_alphas = [1.0, 3.0, 10.0, 30.0, 100.0]
for alpha in all_alphas:
    degree = 2
    print(f'Degree={degree}, alpha={alpha}')
    preprocessor = build_preprocessor_4(degree)
    model = model_2(alpha)
    score = check_model(X_train, Y_train, X_validate, Y_validate, model, preprocessor)
    if score > best_result:
        best_result = score
        best_params = (degree, alpha)
    print('---')

print(f'The best score: {best_result} for degree={best_params[0]}, alpha={best_params[1]}')

Degree=2, alpha=3
(1168, 80) (1168,)
X_train_scaled shape (1168, 128)
Train R2=0.896
Validate R2=0.872
---
Degree=3, alpha=3
(1168, 80) (1168,)
X_train_scaled shape (1168, 293)
Train R2=0.937
Validate R2=0.770
---
Degree=2, alpha=1.0
(1168, 80) (1168,)
X_train_scaled shape (1168, 128)
Train R2=0.897
Validate R2=0.865
---
Degree=2, alpha=3.0
(1168, 80) (1168,)
X_train_scaled shape (1168, 128)
Train R2=0.896
Validate R2=0.872
---
Degree=2, alpha=10.0
(1168, 80) (1168,)
X_train_scaled shape (1168, 128)
Train R2=0.893
Validate R2=0.882
---
Degree=2, alpha=30.0
(1168, 80) (1168,)
X_train_scaled shape (1168, 128)
Train R2=0.885
Validate R2=0.887
---
Degree=2, alpha=100.0
(1168, 80) (1168,)
X_train_scaled shape (1168, 128)
Train R2=0.869
Validate R2=0.881
---
The best score: 0.8865171410578361 for degree=2, alpha=30.0


## Checkpoint with the best model

In [260]:
def build_best_preprocessor(degree=2):
    numeric_transformer = make_pipeline(
        SimpleImputer(strategy='mean'),
        StandardScaler(),
        PolynomialFeatures(degree=degree, include_bias=False),
        StandardScaler(with_mean=False)
    )
    numeric_transformer_2 = make_pipeline(
        SimpleImputer(strategy='constant', fill_value=0),
        StandardScaler() 
    )
    ordinal_encoder = make_pipeline(
        OrdinalEncoder(categories=ordinal_value_per_feature, 
                                      encoded_missing_value=ordinal_values_order.index('TA')),
        StandardScaler()
    )
    one_hot_encoded = make_pipeline(
        SimpleImputer(strategy='most_frequent'), OneHotEncoder()
    )
    return make_pipeline(
        ColumnTransformer(transformers=[
           ('numeric1', numeric_transformer, selected_numerical_features),
           ('numeric2', numeric_transformer_2, ['MasVnrArea']),
           ('one_hot', one_hot_encoded, one_hot_features),
           ('ordinal', ordinal_encoder, ordinal_features),
        ])
    )

def build_best_model(alpha=30.0):
    return Ridge(alpha=alpha)

In [262]:
best_preprocessor = build_best_preprocessor()
best_model = build_best_model()
check_model(X_train, Y_train, X_validate, Y_validate, best_model, best_preprocessor)

(1168, 80) (1168,)
X_train_scaled shape (1168, 128)
Train R2=0.885
Validate R2=0.887


0.8865171410578361

## F-statistic and p-values

In [272]:
%precision 3
preprocessed_columns = best_preprocessor.get_feature_names_out()
from sklearn.feature_selection import f_regression
X_train_scaled = best_preprocessor.transform(X_train)
f_stat, p_values = f_regression(X_train_scaled,Y_train.to_numpy().reshape(1,-1)[0])
df_stat = pd.DataFrame(data={'fstat': f_stat, 'pvalue': p_values}, index=preprocessed_columns)
pd.set_option('display.max_rows', 200)
df_stat

,fstat,pvalue
numeric1__OverallQual,1972.942506,5.366215e-253
numeric1__YearBuilt,424.541792,1.093365e-80
numeric1__Fireplaces,313.586905,2.496737e-62
numeric1__GarageArea,745.553512,2.566378e-127
numeric1__LotArea,85.208715,1.227376e-19
numeric1__GrLivArea,1213.865919,7.404326e-183
numeric1__YearRemodAdd,380.730830,1.343460e-73
numeric1__TotRmsAbvGrd,486.118067,2.515156e-90
numeric1__WoodDeckSF,136.485858,6.726762e-30
numeric1__OverallQual^2,167.390811,7.105832e-36


## Test dataset evaluation

In [195]:
X_test = pd.read_csv('test.csv')

In [196]:
tmp = X_test[selected_numerical_features + ordinal_features + one_hot_features]

tmp.isnull().any()

OverallQual     False
YearBuilt       False
Fireplaces      False
GarageArea       True
YearRemodAdd    False
TotRmsAbvGrd    False
WoodDeckSF      False
ExterQual       False
ExterCond       False
MSZoning         True
LotShape        False
LandContour     False
LotConfig       False
LandSlope       False
Neighborhood    False
BldgType        False
HouseStyle      False
GarageFinish     True
SaleType         True
dtype: bool

In [197]:
the_best_model = m1
the_best_preprocessor = preprocessor_with_ordinal_1
X_test_scaled = the_best_preprocessor.transform(X_test)
Y_test_predict = the_best_model.predict(X_test_scaled)


ValueError: X has 82 features, but LinearRegression is expecting 84 features as input.